In [1]:
# Quantum
import pennylane as qml
# PyTorch
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
# Numpy, Pandas
import numpy as np
import pandas as pd
# Layer
from kan import KAN
from RNN_block import RNN_block
# Data processing
# from fucntions_1028 import data_seq, train_seq
from fucntions import data_seq, train_seq
from sklearn.preprocessing import MinMaxScaler
# Quantum User-Def Classes
from utils import my_utils
from RNNQE_class import NQE
from NQE_train_class import NQE_Train


In [2]:
# CSV 파일 경로
feature_path = './data/pca/pca_train_data_Perth.csv'
label_path = './data/pca/pca_label_data_Perth.csv'

batch_size = 32


new_size = (len(pd.read_csv(feature_path).values) // batch_size) * batch_size
print(new_size)

# CSV 파일을 읽어 Tensor로 변환
feature_data = torch.tensor(pd.read_csv(feature_path).values[:], dtype=torch.float32)
label_data = torch.tensor(pd.read_csv(label_path).values[:], dtype=torch.float32)

# data_seq 객체 생성
data = data_seq(feature_data, label_data)
train_loader, test_loader = data.split_data(test_ratio = 0.2, batch_size=batch_size, seq_first = False, for_nqe = True, n_sequence=5, seed = 40)

3168
dataset length : 3163
test_size : 608
train_size : 2555


In [3]:
for i, (t, l) in enumerate(train_loader):
    print(i, l.shape)
    break

0 torch.Size([32, 2])


In [4]:
def criterion(pred, label):
    '''
        pred : inner product of two states
        label : label data
    '''
    # print('pred : ', pred.shape)
    # print('label :', label.shape)
    # print('lbl[:, 0][-1] :', label[:, 0][-1])
    # print('2nd term :', (label[:, 0] * label[:, 1] + 1).shape)
    # loss = torch.sum((pred - label[:]) ** 2) / len(pred)
    loss = torch.sum(((pred) - 0.5 * (label[:, 0] * label[:, 1] + 1)) ** 2 ) / len(pred)
    return loss

def accuarcy(pred, label):
    '''
        pred : inner product of two states
        label : label data
    '''
    acc = torch.sum((torch.round(pred) == torch.round(0.5 * (label[:, 0] * label[:, 1] + 1)))) / len(pred)
    return acc

### RNNQE

In [5]:
rnnqe = NQE(n_feature=4, mode='RNN', rnn_sequence=5)

In [6]:
for tr, lbl in train_loader:
    print(tr.shape)
    # print(rnnqe(tr).shape)
    # break

torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([32, 2, 5, 4])
torch.Size([

In [7]:
rnnqe_train = NQE_Train(rnnqe, criterion, train_loader, test_loader, [accuarcy])

In [9]:
trained_rnnqe = rnnqe_train.train(500, seq_first=True)

=====Epoch : 1 =====
 loss : 0.10141 metric0 : 0.92593

 train_loss : 0.10141 test_loss : 0.16877 train_metric : 0.92593 test_metric : 0.77467

=====Epoch : 2 =====
 loss : 0.12347 metric0 : 0.85185

 train_loss : 0.12347 test_loss : 0.14820 train_metric : 0.85185 test_metric : 0.79934

=====Epoch : 3 =====
 loss : 0.21783 metric0 : 0.74074

 train_loss : 0.21783 test_loss : 0.16685 train_metric : 0.74074 test_metric : 0.76645

=====Epoch : 4 =====
 loss : 0.12038 metric0 : 0.88889

 train_loss : 0.12038 test_loss : 0.15142 train_metric : 0.88889 test_metric : 0.78783

=====Epoch : 5 =====
 loss : 0.07806 metric0 : 0.88889

 train_loss : 0.07806 test_loss : 0.16205 train_metric : 0.88889 test_metric : 0.77303

=====Epoch : 6 =====
 loss : 0.10302 metric0 : 0.88889

 train_loss : 0.10302 test_loss : 0.16047 train_metric : 0.88889 test_metric : 0.77796

=====Epoch : 7 =====
 loss : 0.14343 metric0 : 0.81481

 train_loss : 0.14343 test_loss : 0.17082 train_metric : 0.81481 test_metric : 0

In [ ]:
torch.save(trained_rnnqe.state_dict(), "./models/pca_RNNQE_loss220.pth")